### This Notebook correctly finds the values of variables in a county by interpolating the original gridded data into a set of 100 pair of latitude and longitude pair. This pair of latitude and longitude is extracted from the geometry attribute (or column) of a shapefile. The interpolation that is used, is the default linear interpolation.

In [1]:
## conda environment Weather_Prediction

import os
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
pd.set_option('display.max_columns', None)

In [2]:
## get single level variables

PATH_1=os.path.join('..','..','..','Weather_Data','CAMS','2010','2010_single_level_34_variables.nc')
sl_34_variables = xr.open_dataset(PATH_1) ## single level 34 variables
sl_34_variables

<xarray.Dataset> Size: 1GB
Dimensions:     (valid_time: 2920, latitude: 34, longitude: 78)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 23kB 2010-01-01 ... 2010-12-31T21...
  * latitude    (latitude) float64 272B 49.25 48.5 47.75 ... 26.0 25.25 24.5
  * longitude   (longitude) float64 624B -124.8 -124.1 -123.3 ... -67.85 -67.1
Data variables: (12/34)
    u10         (valid_time, latitude, longitude) float32 31MB ...
    v10         (valid_time, latitude, longitude) float32 31MB ...
    d2m         (valid_time, latitude, longitude) float32 31MB ...
    t2m         (valid_time, latitude, longitude) float32 31MB ...
    bcaod550    (valid_time, latitude, longitude) float32 31MB ...
    duaod550    (valid_time, latitude, longitude) float32 31MB ...
    ...          ...
    tc_hno3     (valid_time, latitude, longitude) float32 31MB ...
    tcno2       (valid_time, latitude, longitude) float32 31MB ...
    tc_no       (valid_time, latitude, longitude) float32 31MB ...
    gtco3       (valid_time, latitude, longitude) float32 31MB ...
    tc_pan      (valid_time, latitude, longitude) float32 31MB ...
    tc_c3h8     (valid_time, latitude, longitude) float32 31MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-02-09T12:04 GRIB to CDM+CF via cfgrib-0.9.1...

In [3]:
## resample the data to 1 year average

year_avg = sl_34_variables.resample(valid_time='1YE').mean(dim='valid_time')
year_avg

<xarray.Dataset> Size: 362kB
Dimensions:     (valid_time: 1, latitude: 34, longitude: 78)
Coordinates:
  * latitude    (latitude) float64 272B 49.25 48.5 47.75 ... 26.0 25.25 24.5
  * longitude   (longitude) float64 624B -124.8 -124.1 -123.3 ... -67.85 -67.1
  * valid_time  (valid_time) datetime64[ns] 8B 2010-12-31
Data variables: (12/34)
    u10         (valid_time, latitude, longitude) float32 11kB -0.3614 ... -2...
    v10         (valid_time, latitude, longitude) float32 11kB 0.8339 ... -0....
    d2m         (valid_time, latitude, longitude) float32 11kB 279.4 ... 293.2
    t2m         (valid_time, latitude, longitude) float32 11kB 282.6 ... 297.2
    bcaod550    (valid_time, latitude, longitude) float32 11kB 0.007035 ... 0...
    duaod550    (valid_time, latitude, longitude) float32 11kB 0.001286 ... 0...
    ...          ...
    tc_hno3     (valid_time, latitude, longitude) float32 11kB 2.75e-06 ... 3...
    tcno2       (valid_time, latitude, longitude) float32 11kB 2.103e-06 ... ...
    tc_no       (valid_time, latitude, longitude) float32 11kB 2.779e-07 ... ...
    gtco3       (valid_time, latitude, longitude) float32 11kB 0.007557 ... 0...
    tc_pan      (valid_time, latitude, longitude) float32 11kB 5.297e-06 ... ...
    tc_c3h8     (valid_time, latitude, longitude) float32 11kB 8.853e-07 ... ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-02-09T12:04 GRIB to CDM+CF via cfgrib-0.9.1...

In [4]:
year_avg.to_dataframe() ## note that even though the valid time says, 2010-12-31, its the 1 year average

u10       v10         d2m         t2m  \
valid_time latitude longitude                                               
2010-12-31 49.25    -124.85   -0.361422  0.833904  279.353088  282.595428   
                    -124.10   -0.541201  0.882089  279.304169  282.628510   
                    -123.35   -0.555302  0.611501  279.134491  282.693542   
                    -122.60   -0.210227  0.424706  277.481323  281.310242   
                    -121.85   -0.011852  0.388467  274.916321  279.180908   
...                                 ...       ...         ...         ...   
           24.50    -70.10    -2.823348 -0.486432  293.279236  297.384583   
                    -69.35    -2.810792 -0.403280  293.259338  297.346130   
                    -68.60    -2.786537 -0.349180  293.225677  297.305359   
                    -67.85    -2.755172 -0.329040  293.207367  297.277496   
                    -67.10    -2.708043 -0.324282  293.184784  297.249695   

                               bcaod550  duaod550       lsm            msl  \
valid_time latitude longitude                                                
2010-12-31 49.25    -124.85    0.007035  0.001286  0.701660  101420.789062   
                    -124.10    0.007318  0.001264  0.601920  101449.687500   
                    -123.35    0.007716  0.001229  0.537934  101466.539062   
                    -122.60    0.007573  0.001151  0.704370  101491.859375   
                    -121.85    0.006766  0.001037  0.939940  101523.148438   
...                                 ...       ...       ...            ...   
           24.50    -70.10     0.004766  0.021901  0.000000  101583.593750   
                    -69.35     0.004776  0.022322  0.000000  101588.632812   
                    -68.60     0.004770  0.022553  0.000000  101593.937500   
                    -67.85     0.004753  0.022328  0.000000  101598.851562   
                    -67.10     0.004738  0.022333  0.000000  101604.781250   

                               omaod550           pm1         pm2p5  \
valid_time latitude longitude                                         
2010-12-31 49.25    -124.85    0.061123  4.960009e-09  7.045877e-09   
                    -124.10    0.064260  6.942436e-09  9.199298e-09   
                    -123.35    0.069868  1.074456e-08  1.330551e-08   
                    -122.60    0.070421  1.124619e-08  1.373418e-08   
                    -121.85    0.063998  8.690458e-09  1.067045e-08   
...                                 ...           ...           ...   
           24.50    -70.10     0.045098  3.458749e-09  9.263880e-09   
                    -69.35     0.045269  3.446489e-09  9.232994e-09   
                    -68.60     0.045027  3.409170e-09  9.149161e-09   
                    -67.85     0.044643  3.377217e-09  9.084818e-09   
                    -67.10     0.044594  3.346399e-09  9.013966e-09   

                                       pm10  ssaod550  suaod550            z  \
valid_time latitude longitude                                                  
2010-12-31 49.25    -124.85    1.061672e-08  0.034514  0.029658  3219.000244   
                    -124.10    1.356378e-08  0.031535  0.030517  3208.999268   
                    -123.35    1.915926e-08  0.028818  0.034209  3098.375244   
                    -122.60    1.967291e-08  0.025052  0.034248  5099.983398   
                    -121.85    1.530824e-08  0.019942  0.029484  8774.125000   
...                                     ...       ...       ...          ...   
           24.50    -70.10     1.475355e-08  0.033853  0.045072    -2.364403   
                    -69.35     1.470674e-08  0.033949  0.044847     3.694243   
                    -68.60     1.458632e-08  0.033771  0.044255     4.229411   
                    -67.85     1.449778e-08  0.033647  0.043433    -1.469960   
                    -67.10     1.440250e-08  0.033644  0.042867     2.229338   

                                          sp    aod469

In [5]:
## Lets load the county shapefile

SHAPE_PATH = os.path.join('..','..','..','Shapefiles','county_shapefiles','2010_county_shapefile','gz_2010_us_050_00_500k.shp')
county_gdf = gpd.read_file(SHAPE_PATH)

county_gdf

,GEO_ID,STATE,COUNTY,NAME,LSAD,CENSUSAREA,geometry
0,0500000US01029,01,029,Cleburne,County,560.100,"POLYGON ((-85.38872 33.91304, -85.38088 33.873..."
1,0500000US01031,01,031,Coffee,County,678.972,"POLYGON ((-86.03044 31.61894, -86.00408 31.619..."
2,0500000US01037,01,037,Coosa,County,650.926,"POLYGON ((-86.00928 33.10164, -86.00917 33.090..."
3,0500000US01039,01,039,Covington,County,1030.456,"POLYGON ((-86.34851 30.99434, -86.35023 30.994..."
4,0500000US01041,01,041,Crenshaw,County,608.840,"POLYGON ((-86.14699 31.68045, -86.14711 31.663..."
...,...,...,...,...,...,...,...
3216,0500000US72131,72,131,San Sebastián,Muno,70.423,"POLYGON ((-66.90748 18.25314, -66.90739 18.253..."
3217,0500000US72133,72,133,Santa Isabel,Muno,34.023,"POLYGON ((-66.37968 17.94398, -66.38029 17.943..."
3218,0500000US72137,72,137,Toa Baja,Muno,23.241,"MULTIPOLYGON (((-66.13957 18.46232, -66.13956 ..."
3219,0500000US72139,72,139,Trujillo Alto,Muno,20.764,"POLYGON ((-66.02917 18.37590, -66.02828 18.376..."


In [6]:
geometry_column=county_gdf['geometry']
geometry_column

0       POLYGON ((-85.38872 33.91304, -85.38088 33.873...
1       POLYGON ((-86.03044 31.61894, -86.00408 31.619...
2       POLYGON ((-86.00928 33.10164, -86.00917 33.090...
3       POLYGON ((-86.34851 30.99434, -86.35023 30.994...
4       POLYGON ((-86.14699 31.68045, -86.14711 31.663...
                              ...                        
3216    POLYGON ((-66.90748 18.25314, -66.90739 18.253...
3217    POLYGON ((-66.37968 17.94398, -66.38029 17.943...
3218    MULTIPOLYGON (((-66.13957 18.46232, -66.13956 ...
3219    POLYGON ((-66.02917 18.37590, -66.02828 18.376...
3220    POLYGON ((-66.85229 17.95500, -66.85280 17.955...
Name: geometry, Length: 3221, dtype: geometry

In [7]:
lat_lon=geometry_column.get_coordinates(index_parts=True)
lat_lon

x          y
0    0   -85.388717  33.913044
     1   -85.380885  33.873508
     2   -85.379455  33.866291
     3   -85.377426  33.856047
     4   -85.376403  33.850656
...             ...        ...
3220 202 -66.833718  17.989763
     203 -66.835282  17.988274
     204 -66.835429  17.986323
     205 -66.836682  17.965971
     206 -66.852288  17.955004

[837022 rows x 2 columns]

#### In this step we find the minimum number of latitude (or longitude) coordinates that we can extract among all counties from the geometry attribute of the county shapefile

In [8]:
total_num=[]

for i in range(3221): ## total number of counties is 3221
    longitude= lat_lon.loc[(i), 'x'] # get the longitude values. the number of longitudes will be the same as number of latitudes
    num=len(longitude)
    total_num.append(num)
print(min(total_num))

18


In [9]:
index_val=total_num.index(18)  ## get index of the county that has only 18 number of longitude
print(index_val)
row_index_2 = county_gdf.iloc[index_val] ## get the row values with index index_val 
print(row_index_2)

2637
GEO_ID                                           0500000US48173
STATE                                                        48
COUNTY                                                      173
NAME                                                  Glasscock
LSAD                                                     County
CENSUSAREA                                              900.218
geometry      POLYGON ((-101.776085 32.086925, -101.772008 3...
Name: 2637, dtype: object


#### This state is in Texas and has only 18 pairs of latitude and longitude pair

In [10]:
## we will find the total number of counties with less than 100 pair of longitude and latitude
## first lets get rid of state such as guam, puerto rico

test_county=county_gdf
test_county['STATE']=test_county['STATE'].astype(str).astype(int) ## convert dtype of STATE from str to int
df1=test_county.loc[(test_county['STATE'] <= 60)]
df1

,GEO_ID,STATE,COUNTY,NAME,LSAD,CENSUSAREA,geometry
0,0500000US01029,1,029,Cleburne,County,560.100,"POLYGON ((-85.38872 33.91304, -85.38088 33.873..."
1,0500000US01031,1,031,Coffee,County,678.972,"POLYGON ((-86.03044 31.61894, -86.00408 31.619..."
2,0500000US01037,1,037,Coosa,County,650.926,"POLYGON ((-86.00928 33.10164, -86.00917 33.090..."
3,0500000US01039,1,039,Covington,County,1030.456,"POLYGON ((-86.34851 30.99434, -86.35023 30.994..."
4,0500000US01041,1,041,Crenshaw,County,608.840,"POLYGON ((-86.14699 31.68045, -86.14711 31.663..."
...,...,...,...,...,...,...,...
3138,0500000US56027,56,027,Niobrara,County,2626.037,"POLYGON ((-104.05298 42.85955, -104.05286 42.7..."
3139,0500000US56031,56,031,Platte,County,2084.208,"POLYGON ((-104.77417 42.60996, -104.76422 42.6..."
3140,0500000US56037,56,037,Sweetwater,County,10426.649,"POLYGON ((-109.05008 41.00066, -109.17368 41.0..."
3141,0500000US56043,56,043,Washakie,County,2238.549,"POLYGON ((-107.12892 43.99455, -107.12797 43.9..."


In [11]:
## get rid of Hawaii and Alaska, fips code 15 and 2 respectively

df2=df1.loc[(df1['STATE'] !=2 ) & (df1['STATE'] !=15 )]
df2=df2.reset_index(drop=True) ## reset index
df2

,GEO_ID,STATE,COUNTY,NAME,LSAD,CENSUSAREA,geometry
0,0500000US01029,1,029,Cleburne,County,560.100,"POLYGON ((-85.38872 33.91304, -85.38088 33.873..."
1,0500000US01031,1,031,Coffee,County,678.972,"POLYGON ((-86.03044 31.61894, -86.00408 31.619..."
2,0500000US01037,1,037,Coosa,County,650.926,"POLYGON ((-86.00928 33.10164, -86.00917 33.090..."
3,0500000US01039,1,039,Covington,County,1030.456,"POLYGON ((-86.34851 30.99434, -86.35023 30.994..."
4,0500000US01041,1,041,Crenshaw,County,608.840,"POLYGON ((-86.14699 31.68045, -86.14711 31.663..."
...,...,...,...,...,...,...,...
3104,0500000US56027,56,027,Niobrara,County,2626.037,"POLYGON ((-104.05298 42.85955, -104.05286 42.7..."
3105,0500000US56031,56,031,Platte,County,2084.208,"POLYGON ((-104.77417 42.60996, -104.76422 42.6..."
3106,0500000US56037,56,037,Sweetwater,County,10426.649,"POLYGON ((-109.05008 41.00066, -109.17368 41.0..."
3107,0500000US56043,56,043,Washakie,County,2238.549,"POLYGON ((-107.12892 43.99455, -107.12797 43.9..."


In [12]:
geometry_column_2=df2['geometry']
geometry_column_2

0       POLYGON ((-85.38872 33.91304, -85.38088 33.873...
1       POLYGON ((-86.03044 31.61894, -86.00408 31.619...
2       POLYGON ((-86.00928 33.10164, -86.00917 33.090...
3       POLYGON ((-86.34851 30.99434, -86.35023 30.994...
4       POLYGON ((-86.14699 31.68045, -86.14711 31.663...
                              ...                        
3104    POLYGON ((-104.05298 42.85955, -104.05286 42.7...
3105    POLYGON ((-104.77417 42.60996, -104.76422 42.6...
3106    POLYGON ((-109.05008 41.00066, -109.17368 41.0...
3107    POLYGON ((-107.12892 43.99455, -107.12797 43.9...
3108    POLYGON ((-104.05518 43.76163, -104.05514 43.7...
Name: geometry, Length: 3109, dtype: geometry

In [13]:
lat_lon_2=geometry_column_2.get_coordinates(index_parts=True)
lat_lon_2

x          y
0    0   -85.388717  33.913044
     1   -85.380885  33.873508
     2   -85.379455  33.866291
     3   -85.377426  33.856047
     4   -85.376403  33.850656
...             ...        ...
3108 45 -104.054487  44.180381
     46 -104.054950  43.938090
     47 -104.055077  43.936535
     48 -104.055488  43.853477
     49 -104.055176  43.761633

[794693 rows x 2 columns]

In [14]:
type(lat_lon_2)

pandas.core.frame.DataFrame

In [15]:
total_num_2=[] ## find the number of longitudes in each of of the county

for i in range(3109): ## total number of counties here is 3109
    longitude_2= lat_lon_2.loc[(i), 'x']
    num_2=len(longitude_2)
    total_num_2.append(num_2)

count = 0 ## find the number of county with less than 100 points
for x in total_num_2:
    if x < 100:
        count =count+1


In [16]:
print(count)

796


## Find value of the variables in the county with index 0 in the shape file. We will gather the first 100 points of latitude and longitude. If the number is less than 100, that many numbers of latitude and longitude will be extracted.

In [17]:
longitude= lat_lon.loc[(0), 'x']  # get the longitude values of the index 0
extract_val=len(longitude)        ## find number of longitude that a shapfile has
print(extract_val)

## if number of longitude or latitude is less than 100, that many lat-lon pairs will be extracted, if not, a 100 values
if extract_val < 100:
    extract_val=extract_val
else:
    extract_val=100

longitude=longitude[:extract_val]  ## extract first 100 values or values less than 100
longitude

117


0    -85.388717
1    -85.380885
2    -85.379455
3    -85.377426
4    -85.376403
        ...    
95   -85.585985
96   -85.585201
97   -85.636882
98   -85.637126
99   -85.637012
Name: x, Length: 100, dtype: float64

In [18]:
latitude= lat_lon.loc[(0), 'y']   # get the latitude values
latitude=latitude[:extract_val]    ## extract first 100 values or values less than 100
latitude

0     33.913044
1     33.873508
2     33.866291
3     33.856047
4     33.850656
        ...    
95    33.802095
96    33.845920
97    33.846495
98    33.846497
99    33.875273
Name: y, Length: 100, dtype: float64

In [19]:
lat_list=latitude.tolist()
lon_list=longitude.tolist()

In [20]:
## find the corresponding values of the variables in the finer grid

year_avg_finer= year_avg.interp(longitude=lon_list, latitude=lat_list)
year_avg_finer

<xarray.Dataset> Size: 3MB
Dimensions:     (valid_time: 1, latitude: 100, longitude: 100)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 8B 2010-12-31
  * longitude   (longitude) float64 800B -85.39 -85.38 -85.38 ... -85.64 -85.64
  * latitude    (latitude) float64 800B 33.91 33.87 33.87 ... 33.85 33.85 33.88
Data variables: (12/34)
    u10         (valid_time, latitude, longitude) float64 80kB 0.4204 ... 0.3958
    v10         (valid_time, latitude, longitude) float64 80kB -0.1304 ... -0...
    d2m         (valid_time, latitude, longitude) float64 80kB 282.5 ... 282.6
    t2m         (valid_time, latitude, longitude) float64 80kB 289.6 ... 289.7
    bcaod550    (valid_time, latitude, longitude) float64 80kB 0.006261 ... 0...
    duaod550    (valid_time, latitude, longitude) float64 80kB 0.00776 ... 0....
    ...          ...
    tc_hno3     (valid_time, latitude, longitude) float64 80kB 5.417e-06 ... ...
    tcno2       (valid_time, latitude, longitude) float64 80kB 3.947e-06 ... ...
    tc_no       (valid_time, latitude, longitude) float64 80kB 5.642e-07 ... ...
    gtco3       (valid_time, latitude, longitude) float64 80kB 0.006678 ... 0...
    tc_pan      (valid_time, latitude, longitude) float64 80kB 9.767e-06 ... ...
    tc_c3h8     (valid_time, latitude, longitude) float64 80kB 7.208e-07 ... ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-02-09T12:04 GRIB to CDM+CF via cfgrib-0.9.1...

In [21]:
year_avg_finer.to_dataframe()

u10       v10         d2m         t2m  \
valid_time latitude  longitude                                                
2010-12-31 33.913044 -85.388717  0.420356 -0.130393  282.473041  289.576714   
                     -85.380885  0.421158 -0.131242  282.470112  289.574609   
                     -85.379455  0.421305 -0.131397  282.469577  289.574225   
                     -85.377426  0.421513 -0.131617  282.468818  289.573680   
                     -85.376403  0.421618 -0.131728  282.468435  289.573405   
...                                   ...       ...         ...         ...   
           33.875273 -85.585985  0.401309 -0.108572  282.566113  289.662226   
                     -85.585201  0.401394 -0.108654  282.565836  289.662016   
                     -85.636882  0.395829 -0.103189  282.584097  289.675837   
                     -85.637126  0.395803 -0.103163  282.584183  289.675903   
                     -85.637012  0.395815 -0.103175  282.584143  289.675872   

                                 bcaod550  duaod550       lsm            msl  \
valid_time latitude  longitude                                                 
2010-12-31 33.913044 -85.388717  0.006261  0.007760  0.984951  101709.340644   
                     -85.380885  0.006263  0.007755  0.985027  101709.317263   
                     -85.379455  0.006263  0.007754  0.985041  101709.312995   
                     -85.377426  0.006264  0.007753  0.985060  101709.306936   
                     -85.376403  0.006264  0.007753  0.985070  101709.303882   
...                                   ...       ...       ...            ...   
           33.875273 -85.585985  0.006226  0.007908  0.983149  101709.582494   
                     -85.585201  0.006226  0.007907  0.983157  101709.580122   
                     -85.636882  0.006216  0.007938  0.982676  101709.736496   
                     -85.637126  0.006216  0.007938  0.982673  101709.737234   
                     -85.637012  0.006216  0.007938  0.982675  101709.736889   

                                 omaod550           pm1         pm2p5  \
valid_time latitude  longitude                                          
2010-12-31 33.913044 -85.388717  0.078302  1.325125e-08  1.656675e-08   
                     -85.380885  0.078315  1.326112e-08  1.657737e-08   
                     -85.379455  0.078318  1.326292e-08  1.657931e-08   
                     -85.377426  0.078322  1.326548e-08  1.658206e-08   
                     -85.376403  0.078323  1.326677e-08  1.658345e-08   
...                                   ...           ...           ...   
           33.875273 -85.585985  0.077914  1.302498e-08  1.633236e-08   
                     -85.585201  0.077916  1.302595e-08  1.633340e-08   
                     -85.636882  0.077826  1.296221e-08  1.626504e-08   
                     -85.637126  0.077826  1.296191e-08  1.626472e-08   
                     -85.637012  0.077826  1.296205e-08  1.626487e-08   

                                         pm10  ssaod550  suaod550  \
valid_time latitude  longitude                                      
2010-12-31 33.913044 -85.388717  2.295840e-08  0.006070  0.077958   
                     -85.380885  2.297364e-08  0.006069  0.077966   
                     -85.379455  2.297642e-08  0.006069  0.077967   
                     -85.377426  2.298037e-08  0.006068  0.077969   
                     -85.376403  2.298236e-08  0.006068  0.077970   
...                                       ...       ...       ...   
           33.875273 -85.585985  2.261900e-08  0.006130  0.077854   
                     -85.585201  2.262049e-08  0.006130  0.077854   
                     -85.636882  2.252235e-08  0.006138  0.077810   
                     -85.637126  2.252189e-08  0.006138  0.077810   
                     -85.637012  2.252211e-08  0.006138  0.077810   

                                           z            sp    aod469  \
valid_time latitude  longitude                   

In [22]:
df = year_avg_finer.squeeze(dim="valid_time") ## remove the valid_time dimension
df   ## note that the valid_time will then appear as a column

<xarray.Dataset> Size: 3MB
Dimensions:     (latitude: 100, longitude: 100)
Coordinates:
    valid_time  datetime64[ns] 8B 2010-12-31
  * longitude   (longitude) float64 800B -85.39 -85.38 -85.38 ... -85.64 -85.64
  * latitude    (latitude) float64 800B 33.91 33.87 33.87 ... 33.85 33.85 33.88
Data variables: (12/34)
    u10         (latitude, longitude) float64 80kB 0.4204 0.4212 ... 0.3958
    v10         (latitude, longitude) float64 80kB -0.1304 -0.1312 ... -0.1032
    d2m         (latitude, longitude) float64 80kB 282.5 282.5 ... 282.6 282.6
    t2m         (latitude, longitude) float64 80kB 289.6 289.6 ... 289.7 289.7
    bcaod550    (latitude, longitude) float64 80kB 0.006261 ... 0.006216
    duaod550    (latitude, longitude) float64 80kB 0.00776 0.007755 ... 0.007938
    ...          ...
    tc_hno3     (latitude, longitude) float64 80kB 5.417e-06 ... 5.38e-06
    tcno2       (latitude, longitude) float64 80kB 3.947e-06 ... 3.811e-06
    tc_no       (latitude, longitude) float64 80kB 5.642e-07 ... 5.452e-07
    gtco3       (latitude, longitude) float64 80kB 0.006678 ... 0.006675
    tc_pan      (latitude, longitude) float64 80kB 9.767e-06 ... 9.679e-06
    tc_c3h8     (latitude, longitude) float64 80kB 7.208e-07 ... 7.173e-07
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-02-09T12:04 GRIB to CDM+CF via cfgrib-0.9.1...

In [23]:
df.to_dataframe()

u10       v10         d2m         t2m  bcaod550  \
latitude  longitude                                                          
33.913044 -85.388717  0.420356 -0.130393  282.473041  289.576714  0.006261   
          -85.380885  0.421158 -0.131242  282.470112  289.574609  0.006263   
          -85.379455  0.421305 -0.131397  282.469577  289.574225  0.006263   
          -85.377426  0.421513 -0.131617  282.468818  289.573680  0.006264   
          -85.376403  0.421618 -0.131728  282.468435  289.573405  0.006264   
...                        ...       ...         ...         ...       ...   
33.875273 -85.585985  0.401309 -0.108572  282.566113  289.662226  0.006226   
          -85.585201  0.401394 -0.108654  282.565836  289.662016  0.006226   
          -85.636882  0.395829 -0.103189  282.584097  289.675837  0.006216   
          -85.637126  0.395803 -0.103163  282.584183  289.675903  0.006216   
          -85.637012  0.395815 -0.103175  282.584143  289.675872  0.006216   

                      duaod550       lsm            msl  omaod550  \
latitude  longitude                                                 
33.913044 -85.388717  0.007760  0.984951  101709.340644  0.078302   
          -85.380885  0.007755  0.985027  101709.317263  0.078315   
          -85.379455  0.007754  0.985041  101709.312995  0.078318   
          -85.377426  0.007753  0.985060  101709.306936  0.078322   
          -85.376403  0.007753  0.985070  101709.303882  0.078323   
...                        ...       ...            ...       ...   
33.875273 -85.585985  0.007908  0.983149  101709.582494  0.077914   
          -85.585201  0.007907  0.983157  101709.580122  0.077916   
          -85.636882  0.007938  0.982676  101709.736496  0.077826   
          -85.637126  0.007938  0.982673  101709.737234  0.077826   
          -85.637012  0.007938  0.982675  101709.736889  0.077826   

                               pm1         pm2p5          pm10  ssaod550  \
latitude  longitude                                                        
33.913044 -85.388717  1.325125e-08  1.656675e-08  2.295840e-08  0.006070   
          -85.380885  1.326112e-08  1.657737e-08  2.297364e-08  0.006069   
          -85.379455  1.326292e-08  1.657931e-08  2.297642e-08  0.006069   
          -85.377426  1.326548e-08  1.658206e-08  2.298037e-08  0.006068   
          -85.376403  1.326677e-08  1.658345e-08  2.298236e-08  0.006068   
...                            ...           ...           ...       ...   
33.875273 -85.585985  1.302498e-08  1.633236e-08  2.261900e-08  0.006130   
          -85.585201  1.302595e-08  1.633340e-08  2.262049e-08  0.006130   
          -85.636882  1.296221e-08  1.626504e-08  2.252235e-08  0.006138   
          -85.637126  1.296191e-08  1.626472e-08  2.252189e-08  0.006138   
          -85.637012  1.296205e-08  1.626487e-08  2.252211e-08  0.006138   

                      suaod550            z            sp    aod469    aod550  \
latitude  longitude                                                             
33.913044 -85.388717  0.077958  2763.770852  98368.547493  0.215794  0.176351   
          -85.380885  0.077966  2766.862511  98364.844945  0.215817  0.176368   
          -85.379455  0.077967  2767.426886  98364.169054  0.215821  0.176371   
          -85.377426  0.077969  2768.227917  98363.209745  0.215827  0.176376   
          -85.376403  0.077970  2768.631844  98362.726005  0.215830  0.176378   
...                        ...          ...           ...       ...       ...   
33.875273 -85.585985  0.077854  2662.041100  98490.159370  0.215341  0.176032   
          -85.585201  0.077854  2662.343449  98489.796996  0.215343  0.176033   
          -85.636882  0.077810  2642.412671  98513.684549  0.215201  0.175928   
          -85.637126  0.077810  2642.318572  98513.797328  0.215200  0.175928   
          -85.637012  0.077810  2642.362537  98513.744636  0.215200  0.175928   

                        aod670    aod865   aod1240      tcco   tc_c2h6 

In [24]:
## get only the 100 (or less) pair of latitude and longitude from the lat and lon list from the 100,000 (or less) rows
get_vals=[]

for i in range(0, extract_val):
    row=df.isel(latitude=[i], longitude=[i])
    row_df=row.to_dataframe()
    get_vals.append(row_df)

In [25]:
initial_df=pd.concat(get_vals)
initial_df

,,u10,v10,d2m,t2m,bcaod550,duaod550,lsm,msl,omaod550,pm1,pm2p5,pm10,ssaod550,suaod550,z,sp,aod469,aod550,aod670,aod865,aod1240,tcco,tc_c2h6,tchcho,tc_h2o2,tc_oh,tc_c5h8,tc_ch4,tc_hno3,tcno2,tc_no,gtco3,tc_pan,tc_c3h8,valid_time
latitude,longitude,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
33.913044,-85.388717,0.420356,-0.130393,282.473041,289.576714,0.006261,0.007760,0.984951,101709.340644,0.078302,1.325125e-08,1.656675e-08,2.295840e-08,0.006070,0.077958,2763.770852,98368.547493,0.215794,0.176351,0.134556,0.092146,0.051507,0.000883,0.000003,0.000004,0.000008,4.907537e-09,0.000003,0.009704,0.000005,0.000004,5.642466e-07,0.006678,0.00001,7.208176e-07,2010-12-31
33.873508,-85.380885,0.423498,-0.130217,282.494743,289.608909,0.006266,0.007790,0.985059,101708.945307,0.078266,1.327873e-08,1.660486e-08,2.301008e-08,0.006098,0.078033,2739.935777,98396.785786,0.215909,0.176453,0.134645,0.092224,0.051569,0.000883,0.000003,0.000004,0.000008,4.907104e-09,0.000003,0.009707,0.000005,0.000004,5.641315e-07,0.006675,0.00001,7.204435e-07,2010-12-31
33.866291,-85.379455,0.424081,-0.130180,282.498741,289.614788,0.006267,0.007795,0.985078,101708.873068,0.078259,1.328370e-08,1.661176e-08,2.301943e-08,0.006103,0.078046,2735.568657,98401.959244,0.215929,0.176471,0.134661,0.092238,0.051581,0.000883,0.000003,0.000004,0.000008,4.907025e-09,0.000003,0.009707,0.000005,0.000004,5.641108e-07,0.006674,0.00001,7.203752e-07,2010-12-31
33.856047,-85.377426,0.424913,-0.130125,282.504435,289.623134,0.006268,0.007803,0.985105,101708.770499,0.078250,1.329072e-08,1.662151e-08,2.303266e-08,0.006111,0.078065,2729.361761,98409.311944,0.215958,0.176497,0.134683,0.092258,0.051597,0.000883,0.000003,0.000004,0.000008,4.906914e-09,0.000003,0.009708,0.000005,0.000004,5.640815e-07,0.006674,0.00001,7.202783e-07,2010-12-31
33.850656,-85.376403,0.425349,-0.130091,282.507456,289.627539,0.006269,0.007807,0.985119,101708.716635,0.078245,1.329436e-08,1.662658e-08,2.303951e-08,0.006115,0.078075,2726.074052,98413.206710,0.215973,0.176511,0.134695,0.092268,0.051605,0.000883,0.000003,0.000004,0.000008,4.906854e-09,0.000003,0.009708,0.000005,0.000004,5.640628e-07,0.006673,0.00001,7.202268e-07,2010-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33.802095,-85.585985,0.403585,-0.107715,282.603502,289.725200,0.006234,0.007979,0.983350,101708.910091,0.077837,1.306822e-08,1.639662e-08,2.270512e-08,0.006184,0.078009,2615.814415,98545.095949,0.215577,0.176242,0.134561,0.092282,0.051711,0.000884,0.000003,0.000004,0.000008,4.902860e-09,0.000003,0.009722,0.000005,0.000004,5.476001e-07,0.006670,0.00001,7.170963e-07,2010-12-31
33.845920,-85.585201,0.402310,-0.108309,282.580846,289.687277,0.006229,0.007936,0.983237,101709.310385,0.077885,1.304328e-08,1.635915e-08,2.265501e-08,0.006152,0.077917,2643.795564,98511.839451,0.215438,0.176118,0.134452,0.092184,0.051632,0.000884,0.000003,0.000004,0.000008,4.903525e-09,0.000003,0.009718,0.000005,0.000004,5.484630e-07,0.006673,0.00001,7.176290e-07,2010-12-31
33.846495,-85.636882,0.396523,-0.102954,282.598000,289.700552,0.006219,0.007966,0.982768,101709.473633,0.077797,1.298025e-08,1.629162e-08,2.255805e-08,0.006159,0.077874,2624.586061,98534.880715,0.215300,0.176016,0.134388,0.092160,0.051636,0.000884,0.000003,0.000004,0.000008,4.902726e-09,0.000003,0.009721,0.000005,0.000004,5.446330e-07,0.006673,0.00001,7.170068e-07,2010-12-31


In [26]:
final_df=initial_df.reset_index(drop=True) ## reset index and dont include latitude and longitude as column
final_df

,u10,v10,d2m,t2m,bcaod550,duaod550,lsm,msl,omaod550,pm1,pm2p5,pm10,ssaod550,suaod550,z,sp,aod469,aod550,aod670,aod865,aod1240,tcco,tc_c2h6,tchcho,tc_h2o2,tc_oh,tc_c5h8,tc_ch4,tc_hno3,tcno2,tc_no,gtco3,tc_pan,tc_c3h8,valid_time
0,0.420356,-0.130393,282.473041,289.576714,0.006261,0.007760,0.984951,101709.340644,0.078302,1.325125e-08,1.656675e-08,2.295840e-08,0.006070,0.077958,2763.770852,98368.547493,0.215794,0.176351,0.134556,0.092146,0.051507,0.000883,0.000003,0.000004,0.000008,4.907537e-09,0.000003,0.009704,0.000005,0.000004,5.642466e-07,0.006678,0.00001,7.208176e-07,2010-12-31
1,0.423498,-0.130217,282.494743,289.608909,0.006266,0.007790,0.985059,101708.945307,0.078266,1.327873e-08,1.660486e-08,2.301008e-08,0.006098,0.078033,2739.935777,98396.785786,0.215909,0.176453,0.134645,0.092224,0.051569,0.000883,0.000003,0.000004,0.000008,4.907104e-09,0.000003,0.009707,0.000005,0.000004,5.641315e-07,0.006675,0.00001,7.204435e-07,2010-12-31
2,0.424081,-0.130180,282.498741,289.614788,0.006267,0.007795,0.985078,101708.873068,0.078259,1.328370e-08,1.661176e-08,2.301943e-08,0.006103,0.078046,2735.568657,98401.959244,0.215929,0.176471,0.134661,0.092238,0.051581,0.000883,0.000003,0.000004,0.000008,4.907025e-09,0.000003,0.009707,0.000005,0.000004,5.641108e-07,0.006674,0.00001,7.203752e-07,2010-12-31
3,0.424913,-0.130125,282.504435,289.623134,0.006268,0.007803,0.985105,101708.770499,0.078250,1.329072e-08,1.662151e-08,2.303266e-08,0.006111,0.078065,2729.361761,98409.311944,0.215958,0.176497,0.134683,0.092258,0.051597,0.000883,0.000003,0.000004,0.000008,4.906914e-09,0.000003,0.009708,0.000005,0.000004,5.640815e-07,0.006674,0.00001,7.202783e-07,2010-12-31
4,0.425349,-0.130091,282.507456,289.627539,0.006269,0.007807,0.985119,101708.716635,0.078245,1.329436e-08,1.662658e-08,2.303951e-08,0.006115,0.078075,2726.074052,98413.206710,0.215973,0.176511,0.134695,0.092268,0.051605,0.000883,0.000003,0.000004,0.000008,4.906854e-09,0.000003,0.009708,0.000005,0.000004,5.640628e-07,0.006673,0.00001,7.202268e-07,2010-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.403585,-0.107715,282.603502,289.725200,0.006234,0.007979,0.983350,101708.910091,0.077837,1.306822e-08,1.639662e-08,2.270512e-08,0.006184,0.078009,2615.814415,98545.095949,0.215577,0.176242,0.134561,0.092282,0.051711,0.000884,0.000003,0.000004,0.000008,4.902860e-09,0.000003,0.009722,0.000005,0.000004,5.476001e-07,0.006670,0.00001,7.170963e-07,2010-12-31
96,0.402310,-0.108309,282.580846,289.687277,0.006229,0.007936,0.983237,101709.310385,0.077885,1.304328e-08,1.635915e-08,2.265501e-08,0.006152,0.077917,2643.795564,98511.839451,0.215438,0.176118,0.134452,0.092184,0.051632,0.000884,0.000003,0.000004,0.000008,4.903525e-09,0.000003,0.009718,0.000005,0.000004,5.484630e-07,0.006673,0.00001,7.176290e-07,2010-12-31
97,0.396523,-0.102954,282.598000,289.700552,0.006219,0.007966,0.982768,101709.473633,0.077797,1.298025e-08,1.629162e-08,2.255805e-08,0.006159,0.077874,2624.586061,98534.880715,0.215300,0.176016,0.134388,0.092160,0.051636,0.000884,0.000003,0.000004,0.000008,4.902726e-09,0.000003,0.009721,0.000005,0.000004,5.446330e-07,0.006673,0.00001,7.170068e-07,2010-12-31
98,0.396496,-0.102928,282.598082,289.700616,0.006219,0.007966,0.982766,101709.474397,0.077797,1.297995e-08,1.629130e-08,2.255759e-08,0.006159,0.077874,2624.494892,98534.990065,0.215300,0.176016,0.134388,0.092160,0.051636,0.000884,0.000003,0.000004,0.000008,4.902723e-09,0.000003,0.009721,0.000005,0.000004,5.446149e-07,0.006673,0.00001,7.170039e-07,2010-12-31


In [27]:
mean=final_df.mean() ## find the average of the 100 (or less) values
average_df = pd.DataFrame(mean).T 
county_df= average_df.convert_dtypes() ## the dtype of the mean values will be object, so convert that to float
pd.options.display.float_format = '{:.15f}'.format ## see 15 decimal places of the numbers
county_df

,u10,v10,d2m,t2m,bcaod550,duaod550,lsm,msl,omaod550,pm1,pm2p5,pm10,ssaod550,suaod550,z,sp,aod469,aod550,aod670,aod865,aod1240,tcco,tc_c2h6,tchcho,tc_h2o2,tc_oh,tc_c5h8,tc_ch4,tc_hno3,tcno2,tc_no,gtco3,tc_pan,tc_c3h8,valid_time
0,0.413019434594599,-0.107667534800677,282.703011090698624,289.893417245668104,0.006258739615706,0.008160391066076,0.984092741942983,101706.946097755164374,0.077656936743596,0.000000013209071,0.000000016596175,0.000000022974717,0.006331957315087,0.078413175927836,2494.903586698830168,98688.656328798519098,0.216232524283858,0.176821250124215,0.135063312999248,0.092722597516053,0.052063239229144,0.000883831551561,0.000003354210954,0.000004170064362,0.000008342861307,0.000000004900253,0.000002915968832,0.009737069079883,0.000005400752913,0.000003830581230,0.000000545639453,0.006656034353289,0.000009709419429,0.000000714954464,2010-12-31
